In [1]:
#!/usr/bin/env python3

import paratext
import pandas as pd
import lz4.frame
import gzip
import io
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np

'''
filepath = 'cboe/lz4_test/BTCUSD_order_book_20170627.csv.lz4'
#filepath = 'cboe/lz4_test/BTCUSD_order_book_20170627.csv.gz'
df = pandas.read_csv(io.TextIOWrapper(lz4.frame.open(filepath)))
#df = pandas.read_csv(filepath)
#df = paratext.load_csv_to_pandas(gzip.open(filepath).read())
print((df))
'''

from glob import glob
from plumbum.cmd import rm


In [2]:

df = pq.read_table('cboe/parquet_fills_only_BTCUSD.parquet').to_pandas()
print(df.dtypes)
print(df.shape)


Event ID                        int64
Event Date                     object
Event Time                     object
Event Millis                    int64
Order ID                        int64
Execution Options              object
Event Type                     object
Symbol                         object
Order Type                     object
Side                           object
Limit Price (USD)             float64
Original Quantity (BTC)       float64
Gross Notional Value (USD)    float64
Fill Price (USD)              float64
Fill Quantity (BTC)           float64
Total Exec Quantity (BTC)     float64
Remaining Quantity (BTC)      float64
Avg Price (USD)               float64
dtype: object
(9384110, 18)


In [3]:
#num_samples = df.shape[0]
#num_samples_training = round(num_samples * 0.9)
#num_samples_dev = round(num_samples * 0.05)
#num_samples_test = round(num_samples * 0.05)


In [4]:
#train_set = df.iloc[0:num_samples_training]
#dev_set = df.iloc[num_samples_training:num_samples_training+num_samples_dev]
#test_set = df.iloc[num_samples_training+num_samples_dev:]

In [5]:
#small_set = df.iloc[0:10]
#print(small_set)

In [6]:
def get_max_min_price(fulldata):
  max_price = data[['Fill Price (USD)']].max().item()
  min_price = data[['Fill Price (USD)']].min().item()
  return {
    'max': max_price,
    'min': min_price
  }

def extract_price_features(fulldata):
  data = fulldata[['Fill Price (USD)', 'Side', 'Order Type']].copy()
  length = data.shape[0]
  data['isbuy'] = pd.get_dummies(data['Side'])['buy'].values
  data['ismarket'] = pd.get_dummies(data['Order Type'])['market'].values
  return data[['Fill Price (USD)', 'isbuy', 'ismarket']].iloc[:length - 1].values.astype('float32')[:, None, :]

extract_features = extract_price_features

#small_features = extract_features(small_set)

In [7]:
all_features = extract_features(df)

In [8]:
#print(all_features.shape)

In [9]:
num_samples = all_features.shape[0]
num_samples_training = round(num_samples * 0.9)
num_samples_dev = round(num_samples * 0.05)
num_samples_test = round(num_samples * 0.05)


In [10]:
X_train = all_features[0:num_samples_training, :]
X_dev = all_features[num_samples_training:num_samples_training+num_samples_dev, :]
X_test = all_features[num_samples_training+num_samples_dev:, :]

In [11]:
def extract_y_rawprice(fulldata):
  prices = fulldata[['Fill Price (USD)']].values.astype('float32')
  return np.delete(prices, 0, axis=0)
  #return np.insert(prices, 0, prices[0])

extract_y = extract_y_rawprice

#print(len(extract_y(small_set)))

In [12]:
all_y = extract_y(df)

In [13]:
print(all_features.shape)
print(all_y.shape)
print(all_features.dtype)
print(all_y.dtype)

(9384109, 1, 3)
(9384109, 1)
float32
float32


In [14]:
Y_train = all_y[0:num_samples_training, :]
Y_dev = all_y[num_samples_training:num_samples_training+num_samples_dev, :]
Y_test = all_y[num_samples_training+num_samples_dev:, :]

In [15]:
X_sample = all_features[0:10]
Y_sample = all_y[0:10]
print(X_sample)
print(Y_sample)

[[[242.5    1.     0.  ]]

 [[242.5    0.     1.  ]]

 [[242.95   1.     0.  ]]

 [[242.95   0.     0.  ]]

 [[242.96   0.     0.  ]]

 [[242.96   1.     0.  ]]

 [[242.96   1.     0.  ]]

 [[242.96   0.     0.  ]]

 [[242.96   1.     0.  ]]

 [[242.96   0.     0.  ]]]
[[242.5 ]
 [242.95]
 [242.95]
 [242.96]
 [242.96]
 [242.96]
 [242.96]
 [242.96]
 [242.96]
 [243.  ]]


In [16]:
def event_count(time_series, data_name):
  time_series = time_series[['Fill Price (USD)']].values
  upevents = 0
  downevents = 0
  sameprice = 0
  prev_obv = time_series[0]
  for obv in time_series[1:]:
      if obv > prev_obv:
          upevents += 1
      elif obv < prev_obv:
          downevents += 1
      elif obv == prev_obv:
          sameprice += 1
      prev_obv = obv
  print('=== Event counts on %s ===' % data_name)
  print('upevents')
  print(upevents)
  print('downevents')
  print(downevents)
  print('sameprice')
  print(sameprice)
  print()

def mse(time_series, data_name):
  time_series = time_series[['Fill Price (USD)']].values
  total_squared_error = 0
  total_absolute_error = 0
  prev_obv = time_series[0]
  for obv in time_series[1:]:
    total_squared_error += (obv - prev_obv)**2
    total_absolute_error += abs(obv - prev_obv)
    prev_obv = obv
  num_predictions = len(time_series) - 1
  mean_squared_error = total_squared_error / num_predictions
  mean_absolute_error = total_absolute_error / num_predictions
  root_mean_squared_error = np.sqrt(mean_squared_error)
  print('=== baseline on %s ===' % data_name)
  print('total squared error')
  print(total_squared_error)
  print('total absolute error')
  print(total_absolute_error)
  print('mean squared error')
  print(mean_squared_error)
  print('mean absolute error')
  print(mean_absolute_error) 
  print('root mean squared error')
  print(root_mean_squared_error) 
  print()

In [17]:
def show_summary_statistics():
  #event_count(small_set, 'small')
  event_count(train_set, 'train')
  event_count(dev_set, 'dev')
  event_count(test_set, 'test')
  mse(train_set, 'train')
  mse(dev_set, 'dev')
  mse(test_set, 'test')

#show_summary_statistics()

In [18]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import regularizers


Using TensorFlow backend.


In [19]:
def plot_losses(model_history, title):
  plt.figure()
  plt.plot(model_history.history['loss'], label='Train')
  plt.plot(model_history.history['val_loss'], label='Dev')
  plt.xlabel('Epochs'); plt.ylabel('Loss (mse)')
  plt.title(title)
  plt.legend(); plt.show()

In [20]:
def plot_predictions(model, X_test, Y_test, title, inverse=False, scaler=None):
  y_hat = model.predict(X_test)

  if inverse:
      y_hat = inverse_transform(y_hat, scaler)
      Y_test = inverse_transform(Y_test, scaler)

  plt.plot(y_hat, label='Predicted')
  plt.plot(Y_test, label='True')
  plt.xlabel('Time'); 

  if inverse:
      plt.ylabel('Price')
  else:
      plt.ylabel('RESCALED Price')

  plt.title(title)
  plt.legend(); plt.show()

In [21]:
def calculate_MSE_RMSE(model, scaler, X_test, Y_test, model_name):
  y_hat = model.predict(X_test)
  y_hat_inverse = inverse_transform(y_hat, scaler)
  Y_test_inverse = inverse_transform(Y_test, scaler)
  mse = mean_squared_error(Y_test_inverse, y_hat_inverse)
  rmse = np.sqrt(mean_squared_error(Y_test_inverse, y_hat_inverse))
  print('%s:' % model_name)
  print('Test MSE: %.3f' % mse)
  print('Test RMSE: %.3f' % rmse)
  print()

In [22]:
def train_evaluate(model, model_name, 
                   X_train, Y_train, X_dev, Y_dev, X_test, Y_test,
                   lag=10, batch_size=100, epochs=10, verbose=1):

  # Train model
  history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      validation_split=0.05, verbose=verbose, shuffle=False)

  # Plot losses, predictions, and calculate MSE and RMSE
  plot_losses(history, 'Loss\n(%s)' % model_name)
  plot_predictions(model, X_dev, Y_dev, 'Test Predictions\n(%s)' % model_name)
  plot_predictions(model, X_dev, Y_dev, 'Test Predictions\n(%s)' % model_name, inverse=True, scaler=scaler)
  #calculate_MSE_RMSE(model, scaler, X_dev, Y_dev, '%s' % model_name)

In [23]:
def evaluate_test(model, model_name, 
                   X_train, Y_train, X_dev, Y_dev, X_test, Y_test,
                   lag=10, batch_size=100, epochs=10, verbose=1):
    
  # Train model
  #history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
  #                    validation_split=0.05, verbose=verbose, shuffle=False)

  # Plot losses, predictions, and calculate MSE and RMSE
  #plot_losses(history, 'Loss\n(%s)' % model_name)
  plot_predictions(model, X_test, Y_test, 'Test Predictions\n(%s)' % model_name)
  plot_predictions(model, X_test, Y_test, 'Test Predictions\n(%s)' % model_name, inverse=True, scaler=scaler)
  calculate_MSE_RMSE(model, scaler, X_test, Y_test, '%s' % model_name)

In [24]:
# HYPERPARAMETERS
#####################
lag = 1
loss = 'mean_squared_error'
optimizer = 'adagrad' # sgd, adagrad, adam, rmsprop, adagrad
batch_size = 100
epochs = 20
model_name = 'model_LAG-%s_LOSS-%s_OPT-%s_BATCHSIZE-%s_EPOCHS-%s' % (lag, loss, optimizer, batch_size, epochs)
#####################


LSTM_input_shape = [X_train.shape[1], X_train.shape[2]]

In [25]:
# DEFINE MODEL
model = Sequential()

##################### 
model.add(LSTM(200, input_shape=LSTM_input_shape, return_sequences=True))
model.add(Dropout(0.1))

#model.add(LSTM(50, input_shape=LSTM_input_shape, return_sequences=True))
#model.add(Dropout(0.1))

#model.add(LSTM(200, input_shape=LSTM_input_shape, return_sequences=True))
#model.add(Dropout(0.1))

#model.add(LSTM(200, input_shape=LSTM_input_shape, return_sequences=True))
#model.add(Dropout(0.1))

model.add(LSTM(200, input_shape=LSTM_input_shape, return_sequences=True))
model.add(Dropout(0.1))

#model.add(LSTM(256, input_shape=LSTM_input_shape, return_sequences=True))

model.add(LSTM(200, return_sequences=False))
#model.add(Dropout(0.1))

model.add(Dense(1))
#model.add(Dense(1, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))

model.add(Activation('linear'))
#####################

model.compile(loss=loss, optimizer=optimizer)

In [26]:
print(X_train.shape)
print(Y_train.shape)
print(X_train)

(8445698, 1, 3)
(8445698, 1)
[[[2.42500e+02 1.00000e+00 0.00000e+00]]

 [[2.42500e+02 0.00000e+00 1.00000e+00]]

 [[2.42950e+02 1.00000e+00 0.00000e+00]]

 ...

 [[6.01188e+03 0.00000e+00 0.00000e+00]]

 [[6.01188e+03 1.00000e+00 0.00000e+00]]

 [[6.01188e+03 0.00000e+00 0.00000e+00]]]


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3635582174957065690
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7937798964
locality {
  bus_id: 1
}
incarnation: 5197763065719642757
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
# Train/evaluate model
train_evaluate(model, model_name,
               X_train, Y_train, X_dev, Y_dev, X_test, Y_test,
               lag=lag, batch_size=batch_size, epochs=epochs, verbose=1)

Train on 8023413 samples, validate on 422285 samples
Epoch 1/20
8022800/8023413 [============================>.] - ETA: 0s - loss: 10880934.8458